In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer as vectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = pd.read_csv('/content/FakeNewsNet.csv')

In [ ]:
df

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1
...,...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52,1
23192,Zayn Malik & Gigi Hadidâ€™s Shocking Split: Wh...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7,0
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26,1
23194,"Tristan Thompson Feels ""Dumped"" After KhloÃ© K...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24,0


In [ ]:
df.drop(['news_url'],axis=1,inplace=True)

In [ ]:
df.isnull().sum()

title              0
source_domain    330
tweet_num          0
real               0
dtype: int64

In [ ]:
df.dropna(axis=0,inplace=True)
df

,title,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,www.zerchoo.com,38,1
...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,www.express.co.uk,52,1
23192,Zayn Malik & Gigi Hadidâ€™s Shocking Split: Wh...,hollywoodlife.com,7,0
23193,Jessica Chastain Recalls the Moment Her Mother...,www.justjared.com,26,1
23194,"Tristan Thompson Feels ""Dumped"" After KhloÃ© K...",www.intouchweekly.com,24,0


In [ ]:
source = df['source_domain'].values
source

array(['toofab.com', 'www.today.com', 'www.etonline.com', ...,
       'www.justjared.com', 'www.intouchweekly.com', 'www.billboard.com'],
      dtype=object)

In [ ]:
source.shape

(22866,)

In [ ]:
type(source[0])

str

In [ ]:
source_breakdown = source
for index, words in enumerate(source):
  words = words.split(".")
  for word in words:
    if len(word)>3:
      main_word = word
    else:
      pass
  source_breakdown[index] = main_word.upper()


In [ ]:
source_breakdown.shape

(22866,)

In [ ]:
source_breakdown

array(['TOOFAB', 'TODAY', 'ETONLINE', ..., 'JUSTJARED', 'INTOUCHWEEKLY',
       'BILLBOARD'], dtype=object)

In [ ]:
df['source_domain'] = source_breakdown

In [ ]:
title = df['title'].values
sentence = title
for index, words in enumerate(title):
  sentence[index] = words.lower()
sentence

array(["kandi burruss explodes over rape accusation on 'real housewives of atlanta' reunion (video)",
       "people's choice awards 2018: the best red carpet looks",
       "sophia bush sends sweet birthday message to 'one tree hill' co-star hilarie burton: 'breyton 4eva'",
       ...,
       "jessica chastain recalls the moment her mother's boyfriend slapped her: 'i just kicked him in the genitals'",
       'tristan thompson feels "dumped" after khloã© kardashian refuses to let him move into la home (exclusive)',
       "kelly clarkson performs a medley of kendrick lamar's 'humble' & more hits at the billboard music awards"],
      dtype=object)

In [ ]:
df['title'] = sentence
df


,title,source_domain,tweet_num,real
0,kandi burruss explodes over rape accusation on...,TOOFAB,42,1
1,people's choice awards 2018: the best red carp...,TODAY,0,1
2,sophia bush sends sweet birthday message to 'o...,ETONLINE,63,1
3,colombian singer maluma sparks rumours of inap...,DAILYMAIL,20,1
4,gossip girl 10 years later: how upper east sid...,ZERCHOO,38,1
...,...,...,...,...
23191,pippa middleton wedding: in case you missed it...,EXPRESS,52,1
23192,zayn malik & gigi hadidâ€™s shocking split: wh...,HOLLYWOODLIFE,7,0
23193,jessica chastain recalls the moment her mother...,JUSTJARED,26,1
23194,"tristan thompson feels ""dumped"" after khloã© k...",INTOUCHWEEKLY,24,0


In [ ]:
df['content'] = df['source_domain'] + " " + df['title']

In [ ]:
df

,title,source_domain,tweet_num,real,content
0,kandi burruss explodes over rape accusation on...,TOOFAB,42,1,TOOFAB kandi burruss explodes over rape accusa...
1,people's choice awards 2018: the best red carp...,TODAY,0,1,TODAY people's choice awards 2018: the best re...
2,sophia bush sends sweet birthday message to 'o...,ETONLINE,63,1,ETONLINE sophia bush sends sweet birthday mess...
3,colombian singer maluma sparks rumours of inap...,DAILYMAIL,20,1,DAILYMAIL colombian singer maluma sparks rumou...
4,gossip girl 10 years later: how upper east sid...,ZERCHOO,38,1,ZERCHOO gossip girl 10 years later: how upper ...
...,...,...,...,...,...
23191,pippa middleton wedding: in case you missed it...,EXPRESS,52,1,EXPRESS pippa middleton wedding: in case you m...
23192,zayn malik & gigi hadidâ€™s shocking split: wh...,HOLLYWOODLIFE,7,0,HOLLYWOODLIFE zayn malik & gigi hadidâ€™s shoc...
23193,jessica chastain recalls the moment her mother...,JUSTJARED,26,1,JUSTJARED jessica chastain recalls the moment ...
23194,"tristan thompson feels ""dumped"" after khloã© k...",INTOUCHWEEKLY,24,0,"INTOUCHWEEKLY tristan thompson feels ""dumped"" ..."


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
df['content'] = df['content'].apply(stemming)

In [ ]:
X = df['content'].values
Y = df["real"].values

In [ ]:
print(X)

['toofab kandi burruss explod rape accus real housew atlanta reunion video'
 'today peopl choic award best red carpet look'
 'etonlin sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva'
 ...
 'justjar jessica chastain recal moment mother boyfriend slap kick genit'
 'intouchweekli tristan thompson feel dump khlo kardashian refus let move la home exclus'
 'billboard kelli clarkson perform medley kendrick lamar humbl hit billboard music award']


In [ ]:
print(Y)

[1 1 1 ... 1 0 1]


In [ ]:
vectorizer = vectorizer()
vectorizer.fit(X)


X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 13363)	0.21203812761532045
  (0, 12678)	0.29134087462425107
  (0, 10315)	0.2550448920621714
  (0, 10028)	0.2319456304144583
  (0, 9976)	0.3028041959510503
  (0, 6500)	0.3884950982846066
  (0, 5761)	0.26172165902104433
  (0, 4102)	0.3559277141062849
  (0, 1667)	0.3884950982846066
  (0, 631)	0.3080431765216314
  (0, 58)	0.26017883817888826
  (1, 12644)	0.37033078915189493
  (1, 10095)	0.38071955109834293
  (1, 9244)	0.23509521161202573
  (1, 7262)	0.33613527689708445
  (1, 2165)	0.42677718909872997
  (1, 1871)	0.39133670467083703
  (1, 1074)	0.34197150666086185
  (1, 707)	0.31212713786930646
  (2, 12813)	0.29962349676709343
  (2, 12104)	0.22928464107281998
  (2, 11732)	0.14635485037129445
  (2, 11520)	0.27983734064437205
  (2, 10946)	0.2617398000090265
  (2, 8825)	0.19958997043069013
  :	:
  (22863, 1404)	0.24754430987928674
  (22864, 12848)	0.2730129928422586
  (22864, 12521)	0.2714036108097896
  (22864, 10140)	0.320374297714775
  (22864, 8195)	0.28370324871300207
  (22864, 7066)	

In [ ]:
print(X.shape)

(22866, 14098)


In [ ]:
mean = X.mean()
max = X.max()
mean = np.full((22866, 14098),mean)
max = np.full((22866, 14098),max)



X1 = (X - mean)


In [ ]:
X1 = X1/max

In [ ]:
X2 = df['tweet_num'].values
mean = X2.mean()
max = X2.max()
mean =mean = np.full((22866),mean)
max = np.full((22866),max)
X2 = X2 - mean

In [ ]:
X2 = X2/max

In [ ]:
print(X1.shape)

(22866, 14098)


splitting dataset to test and train

In [ ]:
print(X2)
X2  = X2.reshape(-1,1)

[-0.00159666 -0.00304194 -0.00087401 ... -0.00214724 -0.00221606
 -0.00011696]


In [ ]:
X = np.concatenate((X1,X2),axis =1)


X= np.asarray(X)


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y)

Building Model

In [ ]:
model = LogisticRegression()
lreg = model.fit(X_train,Y_train)


testing Model

In [ ]:
# accuracy score on the training error
Y_train_pred= model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train_pred,Y_train)

In [ ]:
print("Accuracy score on training data is:\n", training_data_accuracy)

Accuracy score on training data is:
 0.8792914935490925


In [ ]:
Y_test_pred = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test_pred,Y_test)

In [ ]:
print("Accuracy on test data is :\n", test_data_accuracy)

Accuracy on test data is :
 0.8618277219064276
